In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Compare pipeline runs with Vertex AI Experiments
* This code simplifies [Compare pipeline runs with Vertex AI Experiments](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_pipeline_runs.ipynb#scrollTo=JAPoU8Sm5E6e)

* [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview).

## Set Configuration

### Install Vertex AI SDK for Python and other required packages

In [1]:
%pip install --upgrade --quiet --user --no-warn-conflicts google-cloud-aiplatform \
                                                          "kfp==2.0.0" \
                                                          setuptools

### Authenticate to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.


In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


In [3]:
# @title Configuration

PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_URI = f"gs://mlops_{PROJECT_ID}_7424"  # @param {type:"string"}

## Set Bucket

In [4]:
# @title Create bucket
! gsutil mb -l $LOCATION -p {PROJECT_ID} $BUCKET_URI

Creating gs://mlops_ai-hangsik_7424/...
ServiceException: 409 A Cloud Storage bucket named 'mlops_ai-hangsik_7424' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [6]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://mlops_ai-hangsik_7424/
No changes made to gs://mlops_ai-hangsik_7424/


##Data Preparation


In [7]:
# @title Download training dataset
DATASET_URI = "gs://cloud-samples-data/ai-platform/iris"
!gsutil cp -r $DATASET_URI $BUCKET_URI

Copying gs://cloud-samples-data/ai-platform/iris/classification/evaluate.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/classification/train.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_data.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_predict.csv [Content-Type=text/csv]...
- [4 files][  5.4 KiB/  5.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-samples-data/ai-platform/iris/iris_target.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_test.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_training.cs

In [8]:
# @title Import libraries and define constants

# General
import time
import uuid

#-----------------------------
import kfp.compiler as compiler
# Pipeline Experiments
import kfp.dsl as dsl
# Vertex AI
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1.types.pipeline_state import PipelineState
from kfp.dsl import Metrics, Model, Output, component

#-----------------------------
import logging
logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

In [9]:
# @title Set Experiments and Pipeline

# Experiments
TASK = "classification"
MODEL_TYPE = "xgboost"
EXPERIMENT_NAME = f"{PROJECT_ID}-{TASK}-{MODEL_TYPE}-{uuid.uuid1()}"

# Pipeline
PIPELINE_TEMPLATE_FILE = "pipeline.json"
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/iris/iris_data.csv"
LABEL_URI = f"{BUCKET_URI}/iris/iris_target.csv"
MODEL_URI = f"{BUCKET_URI}/model"

## Training

In [10]:
# @title Initialize Vertex AI SDK for Python
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### Set pre-built containers

* [Pre-built containers for training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers).
* [Pre-built containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers).

In [11]:
TRAIN_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="xgboost", framework_version="1.1", accelerator="cpu"
)

## Formalize the training as pipeline component


Before you start running your pipeline experiments, you have to formalize your training as pipeline component.

To do that, build the pipeline by using the `kfp.dsl.component` decorator to convert your training task into a pipeline component. The below example specifies a base image for the component (python:3.8).

In [12]:
@component(
    base_image="python:3.8",
    packages_to_install=[
        "numpy==1.18.5",
        "pandas==1.0.4",
        "scikit-learn==0.23.1",
        "xgboost==1.1.1",
    ],
)
def custom_trainer(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
    metrics: Output[Metrics],
    model_metadata: Output[Model],
):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split

    # variables
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    def get_logger():
        """
        Get the logger
        """
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, label_path: str
    ) -> (xgb.DMatrix, pd.DataFrame, pd.DataFrame):
        """
        Get the data
        Args:
            train_path: the path of the train data
            label_path: the path of the label data
        Returns:
            the train data and the label data
        """
        # Load data into pandas, then use `.values` to get NumPy arrays
        data = pd.read_csv(train_path).values
        labels = pd.read_csv(label_path).values

        # Convert one-column 2D array into 1D array for use with XGBoost
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(
            data, labels, test_size=0.2, random_state=7
        )

        # Load data into DMatrix object
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        return dtrain, test_data, test_labels

    def train_model(max_depth: int, eta: int, boost_rounds, dtrain: xgb.DMatrix):
        """
        Train the model
        Args:
            max_depth: the max depth of the model
            eta: the eta of the model
            boost_rounds: the boost rounds of the model
            dtrain: the train data
        Returns:
            the trained model
        """
        # Train XGBoost model
        param = {"max_depth": max_depth, "eta": eta}
        model = xgb.train(param, dtrain, num_boost_round=boost_rounds)
        return model

    def evaluate_model(model, test_data, test_labels):
        """
        Evaluate the model
        Args:
            model: the trained model
            test_data: the test data
            test_labels: the test labels
        Returns:
            the accuracy of the model
        """
        dtest = xgb.DMatrix(test_data)
        pred = model.predict(dtest)
        predictions = [round(value) for value in pred]
        # Evaluate predictions
        accuracy = accuracy_score(test_labels, predictions)
        return accuracy

    def save_model(model, model_path):
        """
        Save the model
        Args:
            model: the trained model
            model_path: the path of the model
        """
        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}/model.bst"
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save_model(model_path)

    # Main ----------------------------------------------

    dtrain, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(max_depth, learning_rate, boost_rounds, dtrain)
    accuracy = evaluate_model(model, test_data, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------
    metrics.log_metric("accurancy", accuracy)
    model_metadata.uri = model_uri

## Build a pipeline

Next, create the pipelineJob in associated project.

In [13]:
@dsl.pipeline(name="custom-training-pipeline")
def pipeline(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
):

    custom_trainer(
        train_uri=train_uri,
        label_uri=label_uri,
        max_depth=max_depth,
        learning_rate=learning_rate,
        boost_rounds=boost_rounds,
        model_uri=model_uri,
    )

### Compile the pipeline

Next, compile the pipeline to a JSON file.

In [14]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

## Submit and track pipeline runs

### Submit pipeline runs

Now that you have the pipeline, define its training configuration depending on the defined parameters. In the following example, you can see how to submit several pipeline runs.

In [15]:
runs = [
    {"max_depth": 4, "learning_rate": 0.2, "boost_rounds": 10},
    # {"max_depth": 5, "learning_rate": 0.3, "boost_rounds": 20},
    # {"max_depth": 3, "learning_rate": 0.1, "boost_rounds": 30},
    # {"max_depth": 6, "learning_rate": 0.5, "boost_rounds": 40},
    # {"max_depth": 5, "learning_rate": 0.4, "boost_rounds": 30},
]

In [16]:
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=PIPELINE_TEMPLATE_FILE,
        pipeline_root=PIPELINE_URI,
        parameter_values={
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            **run,
        },
    )
    job.submit(experiment=EXPERIMENT_NAME)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/721521243942/locations/us-central1/pipelineJobs/custom-training-pipeline-20250101011235
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/721521243942/locations/us-central1/pipelineJobs/custom-training-pipeline-20250101011235')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20250101011235?project=721521243942
INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/pipelineJobs/custom-training-pipeline-20250101011235 to Experiment: ai-hangsik-classification-xgboost-708f479a-c7dd-11ef-ae03-0242ac1c000c


### Check Pipeline run states

Vertex AI SDK provides you `get_experiment_df` method to monitor the status of pipeline runs. You can use it either to return parameters and metrics of the pipeline runs in the Vertex AI Experiment or in combination with `get` method of `PipelineJob` to return the pipeline job in Vertex AI Pipeline.


In [17]:
# see state of all pipelineJob
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.boost_rounds,param.train_uri,param.label_uri,param.model_uri,param.max_depth,param.learning_rate,metric.accurancy
0,ai-hangsik-classification-xgboost-708f479a-c7d...,custom-training-pipeline-20250101011235,system.PipelineRun,COMPLETE,10.0,gs://mlops_ai-hangsik_7424/iris/iris_data.csv,gs://mlops_ai-hangsik_7424/iris/iris_target.csv,gs://mlops_ai-hangsik_7424/model,4.0,0.2,0.9


The pipeline runs in the Vertex AI Experiment is monitored based on pipeline run status.

In [18]:
while True:
    pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
    if any(
        pipeline_state != "COMPLETE" for pipeline_state in pipeline_experiments_df.state
    ):
        print("Pipeline runs are still running...")
        if any(
            pipeline_state == "FAILED"
            for pipeline_state in pipeline_experiments_df.state
        ):
            print("At least one Pipeline run failed")
            break
    else:
        print("Pipeline experiment runs have completed")
        break
    time.sleep(60)

Pipeline experiment runs have completed


In [19]:
# Get the PipelineJob resource using the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[0])
print("Pipeline job name: ", job.resource_name)
print("Pipeline Run UI link: ", job._dashboard_uri())

Pipeline job name:  projects/721521243942/locations/us-central1/pipelineJobs/custom-training-pipeline-20250101011235
Pipeline Run UI link:  https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20250101011235?project=721521243942


## Load Model

In [52]:
# @title Load the model from GCS
import pickle
import xgboost as xgb

from google.cloud import storage
import io
# Initialize a Cloud Storage client
storage_client = storage.Client(project=PROJECT_ID)

BUCKET_NAME = "mlops_ai-hangsik_7424"
MODEL_FILE = "model/a77d55bc-c7dd-11ef-9b0b-661500983924/model.bst"

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(MODEL_FILE)

# Download the model file to a local temporary file
local_model_path = "./model.bst"
blob.download_to_filename(local_model_path)

# Load the XGBoost model using xgboost.Booster.load_model
model = xgb.Booster()  # Create a Booster object
model.load_model(local_model_path) # Load the model from the t


In [53]:
# @title Predict
dtest = xgb.DMatrix([[1.1, 4.5, 1.4, 2.2]])  # Example input features
prediction = model.predict(dtest)

print(f"Prediction: {prediction}")

Prediction: [0.05681667]


In [55]:
# @title Predict

# Prepare new data for prediction
"""
5.4,3.9,1.7,0.4,0
7.7,3.8,6.7,2.2,2
6.3,3.3,4.7,1.6,1
6.8,3.2,5.9,2.3,2
7.6,3.0,6.6,2.1,2
6.4,3.2,5.3,2.3,2

"""

new_data = [[6.3,3.3,4.7,1.6]]
dnew = xgb.DMatrix(new_data)

# Make predictions
predictions = model.predict(dnew)

# Get predicted class labels (0, 1, or 2 for Iris species)
predicted_labels = [int(round(pred)) for pred in predictions]

print(f"Predicted labels: {predicted_labels}")


Predicted labels: [1]


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial

In [ ]:
# Delete the pipeline
while True:
    for i in range(0, len(runs)):
        pipeline_job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[i])
        if pipeline_job.state != PipelineState.PIPELINE_STATE_SUCCEEDED:
            print("Pipeline job is still running...")
            time.sleep(60)
        else:
            print("Pipeline job is complete.")
            pipeline_job.delete()
    break

# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion
if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

# Remove local files
!rm {PIPELINE_TEMPLATE_FILE}